In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3)
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [ ]:
# Load Titanic dataset 
df = pd.read_csv('C:/Users/kdehn/OneDrive - stevens.edu/___Data/CSV_files/Titanic_rows.csv')

print(df[df.isna().any(axis=1)]) # shows NaN values in data frame if it exists
df.info()

In [ ]:
df_num= df.select_dtypes(exclude='object')
df_obj= df.select_dtypes(include='object')
df_num.info()
df_obj.info()

In [ ]:
df_obj= pd.get_dummies(df_obj, drop_first=True)
print(df_obj['Survived_Yes'].value_counts())
print(df_obj['Survived_Yes'].value_counts())
print( (df_obj['Survived_Yes'].value_counts()/ df_obj['Survived_Yes'].value_counts().sum()) * 100)
sns.countplot(data=df_obj, x='Survived_Yes')
df_obj.head()

In [ ]:
attr = df_obj.drop('Survived_Yes', axis=1)
target = df_obj['Survived_Yes']

# Split the data into training and testing sets
attr_train, attr_test, target_train, target_test = train_test_split(attr, target, test_size=0.2, random_state=77)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100,criterion='entropy', random_state=77)
# model = RandomForestClassifier(n_estimators=100,criterion='gini', random_state=7)


In [ ]:
model.fit(attr_train,target_train)
target_pred = model.predict(attr_test)
 

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(f"Accuracy= {accuracy_score(target_test,target_pred)}")

print(confusion_matrix(target_test,target_pred))
print(classification_report(target_test,target_pred))

In [ ]:

# Feature importance for Mean Decrease Accuracy
feature_importance = model.feature_importances_
# # Feature importance for Mean Decrease GINI or any impuritiy
 
print(f"feature importance: {feature_importance}" )



In [ ]:
plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_importance)), feature_importance, align='center')
plt.yticks(range(len(feature_importance)), attr.columns)
plt.xlabel('Mean decrease in impurity')
plt.title('Feature Importance - Mean decrease in impurity')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [ ]:
# Calculate initial accuracy
initial_accuracy = accuracy_score(target_test,target_pred)
print(initial_accuracy)

In [ ]:
feature_importances = []

# Calculate Mean Decrease Accuracy for each feature

for i in range(attr_test.shape[1]):
    attr_test_permuted = attr_test.copy()
    np.random.shuffle(attr_test_permuted.iloc[:, i].values)
    permuted_accuracy = accuracy_score(target_test, model.predict(attr_test_permuted))
    feature_importances.append(initial_accuracy - permuted_accuracy)
    print( attr_test_permuted.columns[i],f" reduction in accuracy {initial_accuracy - permuted_accuracy}")

# Create a DataFrame to display feature importances
feature_importance_df = pd.DataFrame({'Feature': attr.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel('Mean Decrease Accuracy')
plt.ylabel('Feature')
plt.title('Feature Importance based on Mean Decrease Accuracy')
plt.show()

print(feature_importance_df)

In [ ]:
single_tree = model.estimators_[1]
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

plt.figure(figsize=(50,30), dpi=250)
plot_tree(single_tree,fontsize=20,filled=True,feature_names=attr.columns);